In [213]:
import cv2
import numpy as np


In [214]:
def pyrDown_custom(imagen, sigma):
    filtro = crear_filtro_gaussiano(sigma)
    imagen = cv2.filter2D(imagen, -1, filtro)
    imagen = cv2.resize(imagen, (0,0), fx=0.5, fy=0.5)
    return imagen

def crear_filtro_gaussiano(sigma):
    filter_size = 2 * int(4 * sigma + 0.5) + 1
    gaussian_filter = np.zeros((filter_size, filter_size), np.float32)
    m = filter_size//2
    n = filter_size//2

    for x in range(-m, m+1):
        for y in range(-n, n+1):
            x1 = 2*np.pi*(sigma**2)
            x2 = np.exp(-(x**2 + y**2)/(2* sigma**2))
            gaussian_filter[x+m, y+n] = (1/x1)*x2
    print(gaussian_filter)
    return gaussian_filter

# Convolucion con Fourier
def convolucion_dft(imagen, sigma):
    imagen = imagen.copy()
    if sigma:
      filtro = crear_filtro_gaussiano(sigma)


    # Aplicar la DFT bidimensional a la imagen y al filtro
    dft_imagen = np.fft.fft2(imagen)
    dft_filtro = np.fft.fft2(filtro, s=imagen.shape)

    # Multiplicar punto a punto la DFT de la imagen y la DFT del filtro
    resultado_dft = dft_imagen * dft_filtro

    # Aplicar la Transformada de Fourier Inversa (IDFT) para obtener la imagen convolucionada
    imagen_convolucionada = np.fft.ifft2(resultado_dft).real

    # Escalar la imagen resultante para valores entre 0 y 255
    imagen_convolucionada = np.uint8(imagen_convolucionada)
    imagen_convolucionada = cv2.resize(imagen_convolucionada, (0,0), fx=0.5, fy=0.5)

    return imagen_convolucionada

def convolucion_dft(imagen, sigma):
    imagen = imagen.copy()
    if sigma:
        filtro = crear_filtro_gaussiano(sigma)

    b, g, r = cv2.split(imagen)

    result_r = np.zeros_like(r)
    result_g = np.zeros_like(g)
    result_b = np.zeros_like(b)

    dft_filtro = np.fft.fft2(filtro, s=imagen.shape[:2])
    dft_r = np.fft.fft2(r)
    dft_g = np.fft.fft2(g)
    dft_b = np.fft.fft2(b)

    result_r = np.fft.ifft2(dft_r * dft_filtro).real
    result_g = np.fft.ifft2(dft_g * dft_filtro).real
    result_b = np.fft.ifft2(dft_b * dft_filtro).real

    imagen_convolucionada = cv2.merge((result_b, result_g, result_r))
    imagen_convolucionada = np.uint8(imagen_convolucionada)
    print(" TAMAÑO DE IMAGEN CONV" , imagen_convolucionada.shape)
    # if imagen_convolucionada.shape[0] > 500:
    #     cv2.imwrite("convolucion_dft.jpg", imagen_convolucionada)
    imagen_convolucionada = cv2.resize(imagen_convolucionada, (0, 0), fx=0.5, fy=0.5)

        
    return imagen_convolucionada

# def convolucion_dft(imagen, sigma):
#     if sigma:
#         filtro = crear_filtro_gaussiano(sigma)
#         filtro = np.fft.fft2(filtro, s=imagen.shape[:2])
#         b, g, r = cv2.split(imagen)
#         b_transformada = np.fft.fft2(b)
#         g_transformada = np.fft.fft2(g)
#         r_transformada = np.fft.fft2(r)
#         b_shifted = np.fft.fftshift(b_transformada)
#         g_shifted = np.fft.fftshift(g_transformada)
#         r_shifted = np.fft.fftshift(r_transformada)
#         b_filtered = b_shifted * filtro
#         g_filtered = g_shifted * filtro
#         r_filtered = r_shifted * filtro
#         b_fin = np.fft.ifft2(np.fft.ifftshift(b_filtered)).real
#         g_fin = np.fft.ifft2(np.fft.ifftshift(g_filtered)).real
#         r_fin = np.fft.ifft2(np.fft.ifftshift(r_filtered)).real
#         imagen_convolucionada = cv2.merge((b_fin, g_fin, r_fin))
#         imagen_convolucionada = cv2.normalize(imagen_convolucionada, None, 0, 255, cv2.NORM_MINMAX)
#         imagen_convolucionada = np.uint8(imagen_convolucionada)
#         imagen_convolucionada = cv2.resize(imagen_convolucionada, (0, 0), fx=0.5, fy=0.5)
#     return imagen_convolucionada

def crear_filtro_gaussiano2(sigma, rows, cols):
    x = np.arange(-cols // 2, cols // 2)
    y = np.arange(-rows // 2, rows // 2)
    xx, yy = np.meshgrid(x, y)
    filtro = np.exp(-(xx**2 + yy**2) / (2.0 * sigma**2))
    return filtro / np.sum(filtro)

    

def calcularGauss(imagen, niveles, sigma, fourier = False)->list:
    copia = imagen.copy()
    piramide_gaussiana = [copia]
    for i in range(niveles):
        # Funcion de gauss
        # copia = cv2.pyrDown(copia) #funcion de max gaus -> reshape
        if fourier:
            copia = convolucion_dft(copia, sigma)
        else:
            copia = pyrDown_custom(copia, sigma)
        piramide_gaussiana.append(copia)
    return piramide_gaussiana

def calcularLaplaciana(piramide_gaussiana):
    piramide_laplaciana = []
    for i in range(len(piramide_gaussiana)-1,0, -1):
        g_i = cv2.pyrUp(piramide_gaussiana[i]) #gi es la imagen incrementada en tamaño y la que deberia ser modificada si no hace match en tamaño
        print("i: ", i, "GI: ", g_i.shape, "GI-1", piramide_gaussiana[i-1].shape)
        g_i = ajustar_tamano(piramide_gaussiana[i-1], g_i)
        print("i: ", i, "GI: ", g_i.shape, "GI-1", piramide_gaussiana[i-1].shape)
        l_i = cv2.subtract(piramide_gaussiana[i-1], g_i)
        piramide_laplaciana.append(l_i)
    return piramide_laplaciana

def ajustar_tamano(img_norm, img_x2):
    canales = img_norm.shape[2]
    if img_x2.shape[0] < img_norm.shape[0]:
        empty_column = np.zeros((1, img_x2.shape[1], canales), dtype=np.uint8)
        img_x2 = np.vstack((img_norm, empty_column))
    elif img_x2.shape[0] > img_norm.shape[0]:
        # remove last row
        img_x2 = img_x2[:-1, :, :]
    
    if img_x2.shape[1] < img_norm.shape[1]:
        print("Entro Ajusto fila")
        empty_column = np.zeros((img_x2.shape[0], 1, canales), dtype=np.uint8)
        img_x2 = np.hstack((img_x2, empty_column))
        print(img_x2.shape)
    elif img_x2.shape[1] > img_norm.shape[1]:
        print("Entro Ajusto columna")
        img_x2 = img_x2[:, :-1, :]
    return img_x2

def reconstruir(laplacianas, gaussianas):
    img_r = gaussianas[-1]
    # img_r = cv2.pyrUp(gaussiana_i)
    for i in range(len(gaussianas)):
        print(i)
        try:
            laplaciana_i = laplacianas[-i-1]
        except:
            print("Error", i)
            break
        img_r = cv2.pyrUp(img_r)
        print("laplaciana_i", laplaciana_i.shape, "img_r", img_r.shape)
        img_r = ajustar_tamano(laplaciana_i, img_r)
        img_r = cv2.add(img_r, laplaciana_i)
        cv2.imwrite("./reconstruida/img_reconstruida" + str(i) + ".jpg", img_r)

        

def guardar_imgs(laplacianas, gaussianas, nombre):
    for i in range(len(laplacianas)):
        cv2.imwrite("./piramides_"+ str(nombre)+"/laplaciana" + str(i) + ".jpg", laplacianas[i])
    for i in range(len(gaussianas)):
        cv2.imwrite("./piramides_"+ str(nombre) +"/gaussiana" + str(i) + ".jpg", gaussianas[i])

In [230]:
name_a = "girl"
name_b = "woman"
A = cv2.imread('{}.jpeg'.format(name_a))
B = cv2.imread('{}.jpeg'.format(name_b))

print("Shape A: ", A.shape, "Shape B: ", B.shape)
if A.shape[0]%2 ==1:
    A = A[:-1, :]
if A.shape[1]%2 ==1:
    A = A[:, :-1]
if B.shape[0]%2 ==1:
    B = B[:-1, :]
if B.shape[1]%2 ==1:
    B = B[:, :-1]
print("Shape A: ", A.shape, "Shape B: ", B.shape)

sigma = 0.5
ga = calcularGauss(A, 5, sigma, fourier = False)
gb = calcularGauss(B, 5, sigma, fourier = False)
# ga = calcularGauss(A, 5, sigma, fourier = True)
# gb = calcularGauss(B, 5, sigma, fourier = True)

la = calcularLaplaciana(ga)
lb = calcularLaplaciana(gb)

Shape A:  (338, 282, 3) Shape B:  (338, 282, 3)
Shape A:  (338, 282, 3) Shape B:  (338, 282, 3)
[[7.1642120e-08 2.8902494e-05 2.1356215e-04 2.8902494e-05 7.1642120e-08]
 [2.8902494e-05 1.1660098e-02 8.6157121e-02 1.1660098e-02 2.8902494e-05]
 [2.1356215e-04 8.6157121e-02 6.3661975e-01 8.6157121e-02 2.1356215e-04]
 [2.8902494e-05 1.1660098e-02 8.6157121e-02 1.1660098e-02 2.8902494e-05]
 [7.1642120e-08 2.8902494e-05 2.1356215e-04 2.8902494e-05 7.1642120e-08]]
[[7.1642120e-08 2.8902494e-05 2.1356215e-04 2.8902494e-05 7.1642120e-08]
 [2.8902494e-05 1.1660098e-02 8.6157121e-02 1.1660098e-02 2.8902494e-05]
 [2.1356215e-04 8.6157121e-02 6.3661975e-01 8.6157121e-02 2.1356215e-04]
 [2.8902494e-05 1.1660098e-02 8.6157121e-02 1.1660098e-02 2.8902494e-05]
 [7.1642120e-08 2.8902494e-05 2.1356215e-04 2.8902494e-05 7.1642120e-08]]
[[7.1642120e-08 2.8902494e-05 2.1356215e-04 2.8902494e-05 7.1642120e-08]
 [2.8902494e-05 1.1660098e-02 8.6157121e-02 1.1660098e-02 2.8902494e-05]
 [2.1356215e-04 8.6157121e

error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/core/src/arithm.cpp:650: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'arithm_op'


In [231]:
for i in ga:
    print(i.shape)
print("Laplace")
for i in la:
    print(i.shape)

(338, 282, 3)
(169, 141, 3)
(84, 70, 3)
(42, 35, 3)
(21, 18, 3)
(10, 9, 3)
Laplace
(36, 28, 3)
(71, 55, 3)
(142, 110, 3)
(284, 221, 3)
(568, 442, 3)


In [232]:
print("Gaussiana A: ", len(ga), "Gaussiana B: ", len(gb))
print("Laplaciana A: ", len(la), "Laplaciana B: ", len(lb))
print("Gaussiana A size: ", ga[-1].shape, "Gaussiana B size: ", gb[-1].shape)
print("Laplaciana A size: ", la[-1].shape, "Laplaciana B size: ", lb[-1].shape)

Gaussiana A:  6 Gaussiana B:  6
Laplaciana A:  5 Laplaciana B:  5
Gaussiana A size:  (10, 9, 3) Gaussiana B size:  (10, 9, 3)
Laplaciana A size:  (568, 442, 3) Laplaciana B size:  (568, 442, 3)


In [233]:
reconstruir(la[::-1], gb)

0
laplaciana_i (36, 28, 3) img_r (20, 18, 3)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 28 and the array at index 1 has size 18

In [234]:
guardar_imgs(la, ga, name_a)
guardar_imgs(lb, gb, name_b)

In [102]:
def blend_laplacian_pyramids(laplacian1, laplacian2):
    blended_pyramid = []
    for l1, l2 in zip(laplacian1, laplacian2):
        rows, cols, _ = l1.shape
        # Perform the blending in the frequency domain (Fourier Transform)
        fft_l1 = np.fft.fft2(l1)
        fft_l2 = np.fft.fft2(l2)
        blended = np.fft.ifft2((fft_l1 + fft_l2) / 2).real
        blended_pyramid.append(blended)
    return blended_pyramid

blended_laplacian_pyramid = blend_laplacian_pyramids(la, lb)


In [103]:
reconstruir(blended_laplacian_pyramid[::-1], gb)

0
laplaciana_i (36, 28, 3) img_r (36, 28, 3)


error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/core/src/arithm.cpp:674: error: (-5:Bad argument) When the input arrays in add/subtract/multiply/divide functions have different types, the output array type must be explicitly specified in function 'arithm_op'
